# Low API test of TensorFlow
---
To satisify the purpose of copy all the `variables` from one **nn** to another **nn**
We need to use low API(tensor, variables) to build a **nn** from scratch and test the copy

In [1]:
import tensorflow as tf

### Test the low level TensorFlow API the ability of copying the weights

Unfortunately, the high level of TF cannot copy the weights of NN by value.

Thus, we have to use the low API approach to copy the weights directly

In [2]:
num_actions = 1
num_features = 1
state = tf.constant([[1], [2], [3], [4]], dtype=tf.float32)#tf.placeholder(tf.float32, [None, num_features], name="state")
y_true = tf.constant([[0], [-1], [-2], [-3]], dtype=tf.float32)

with tf.variable_scope("predict_net"):
    collections = ["predict_net_params", tf.GraphKeys.GLOBAL_VARIABLES]
    num_nodes = 10
    # weight_initializer = tf.random_normal_initializer(0, 0.3)
    weight_initializer = tf.constant_initializer(value=1, dtype=tf.float32)
    bias_initializer = tf.constant_initializer(0.1)

    with tf.variable_scope("layer1"):
        layer1_weight = tf.get_variable(name="weight", shape=[num_features, num_nodes],
                                        initializer=weight_initializer, collections=collections)
        layer1_bias = tf.get_variable(name="bias", shape=[1, num_nodes], initializer=bias_initializer,
                                      collections=collections)
        layer1 = tf.nn.relu(tf.matmul(state, layer1_weight) + layer1_bias)
        
    with tf.variable_scope("layer2"):
        layer2_weight = tf.get_variable(name="weight", shape=[num_nodes, num_actions],
                                        initializer=weight_initializer, collections=collections)
        layer2_bias = tf.get_variable(name="bias", shape=[1, num_actions], initializer=bias_initializer,
                                      collections=collections)
        q_predict = tf.nn.relu(tf.matmul(layer1, layer2_weight) + layer2_bias)
    
with tf.variable_scope("target_net"):
    collections = ["target_net_params", tf.GraphKeys.GLOBAL_VARIABLES]
    num_nodes = 10
    # weight_initializer = tf.random_normal_initializer(0, 0.3)
    weight_initializer = tf.zeros_initializer()
    bias_initializer = tf.constant_initializer(0.1)

    with tf.variable_scope("layer1"):
        layer1_weight = tf.get_variable(name="weight", shape=[num_features, num_nodes],
                                        initializer=weight_initializer, collections=collections)
        layer1_bias = tf.get_variable(name="bias", shape=[1, num_nodes], initializer=bias_initializer,
                                      collections=collections)
        layer1 = tf.nn.relu(tf.matmul(state, layer1_weight) + layer1_bias)
        
    with tf.variable_scope("layer2"):
        layer2_weight = tf.get_variable(name="weight", shape=[num_nodes, num_actions],
                                        initializer=weight_initializer, collections=collections)
        layer2_bias = tf.get_variable(name="bias", shape=[1, num_actions], initializer=bias_initializer,
                                      collections=collections)
        q_target = tf.nn.relu(tf.matmul(layer1, layer2_weight) + layer2_bias)

In [3]:
init_op = tf.global_variables_initializer()
sess = tf.Session()
# Initialise the variables( weights & bias ), have to do before any sess.run()
sess.run(init_op)
print(sess.run(q_predict))
print(sess.run(q_target))

[[11.100001]
 [21.1     ]
 [31.1     ]
 [41.099995]]
[[0.1]
 [0.1]
 [0.1]
 [0.1]]


### Test variables assignment by hand

To copy the varibles from predict NN to target NN we need to copy the weights periodically, thus make follow test. High level API for tf cannot perform copy by value, unfortunately.

In [4]:

assignment_tf = tf.constant(value=[1], shape=[11, 11], dtype=tf.float32)
print(sess.run(assignment_tf))

[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]]


In [5]:
test_variable = tf.get_variable(name="test", shape=[11, 11], dtype=tf.float32, initializer=tf.zeros_initializer())


In [6]:
sess.run(test_variable.initializer)

In [7]:

assign_op = tf.assign(ref=test_variable, value=assignment_tf)
sess.run(assign_op)
print(sess.run(test_variable))

[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]]


### Test to assign a value to variables in side of NN and make a prediction

In [12]:
# Noted, here return a list of tf
target_variables = tf.get_collection("target_net_params")
predict_variables = tf.get_collection("predict_net_params")
for predict_variable, target_variable in zip(predict_variables, target_variables):
    print(predict_variable)
    print(target_variable)

# target_variables.assign(predict_variables)
# sess.run(assign_op)
# print(target_variables)

<tf.Variable 'predict_net/layer1/weight:0' shape=(1, 10) dtype=float32_ref>
<tf.Variable 'target_net/layer1/weight:0' shape=(1, 10) dtype=float32_ref>
<tf.Variable 'predict_net/layer1/bias:0' shape=(1, 10) dtype=float32_ref>
<tf.Variable 'target_net/layer1/bias:0' shape=(1, 10) dtype=float32_ref>
<tf.Variable 'predict_net/layer2/weight:0' shape=(10, 1) dtype=float32_ref>
<tf.Variable 'target_net/layer2/weight:0' shape=(10, 1) dtype=float32_ref>
<tf.Variable 'predict_net/layer2/bias:0' shape=(1, 1) dtype=float32_ref>
<tf.Variable 'target_net/layer2/bias:0' shape=(1, 1) dtype=float32_ref>


In [13]:
for predict_variable, target_variable in zip(predict_variables, target_variables):
    assign_op = tf.assign(ref=target_variable, value=predict_variable)
    sess.run(assign_op)
print(sess.run(q_target))

[[11.100001]
 [21.1     ]
 [31.1     ]
 [41.099995]]


### Test training prediction NN won't affect the variables inside target NN
Now after assignment, we just need a further test to ensure that it works


In [14]:
loss = tf.losses.mean_squared_error(labels=y_true, predictions=q_predict)
print(sess.run(loss))

913.0099


In [15]:
optimiser = tf.train.GradientDescentOptimizer(0.001)
train = optimiser.minimize(loss)
for i in range(40):
    _, loss_value = sess.run((train, loss))
    print(loss_value)

913.0099
447.41913
278.79083
194.47527
145.21652
113.55751
91.834
76.194824
64.51275
55.527016
48.448147
42.759583
38.110817
34.25666
31.02114
28.275074
25.921791
23.887686
22.115868
20.561762
19.19001
17.972261
16.88555
15.911096
15.03343
14.239693
13.519127
12.862677
12.262663
11.712551
11.206728
10.740374
10.309311
9.909855
9.538421
9.192185
8.8688
8.566196
8.282532
8.016171


In [16]:
# Train make prediction with 2 nn, we will have different value.
print(sess.run(q_predict))
print(sess.run(q_target))

[[0.        ]
 [0.39366665]
 [0.9546255 ]
 [1.5155843 ]]
[[11.100001]
 [21.1     ]
 [31.1     ]
 [41.099995]]
